Hi there! 👋

Welcome to the second part of the series - Langchain and LLMs for software testing. In part1 of this series we explored the idea of using lrge language models to assist us in generating test cases and in this part we will extend this idea further to see if these models can execute tests for us in an automated way.

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.4 MB/s eta 0:00:00


In [2]:
!pip install pydantic

In [3]:
!pip install typing_extensions==4.6.3

In [4]:
!pip install -U git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-7tsuolbv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-7tsuolbv
  Resolved https://github.com/huggingface/transformers.git to commit 641adca55832ed9c5648f54dcd8926d67d3511db
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7450897 sha256=b1f9f1f0d259e25e59f2f0976a5c57df2c5c0d590265bf19628a27fba771cb48
  Stored in directory: /tmp/pip-ephem-wheel-cache-o45f2i0l/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers

In [5]:
!pip install -U git+https://github.com/huggingface/accelerate.git

  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-32agv9o4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-32agv9o4
  Resolved https://github.com/huggingface/accelerate.git to commit 49cb83a423f2946059117d8bb39b7c8747d29d80
  Installing build dependencies ... - \ | done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=249138 sha256=bc16ded8cf24666557e4a22559579e5fb8cbf2c145290753f16793f78f36dace
  Stored in directory: /tmp/pip-ephem-wheel-cache-h0p908h1/wheels/9c/a3/1e/47368f9b6575655fe9ee1b6350cfa7d4b0befe66a35f8a8365
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


In [6]:
!pip install --upgrade bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 37.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [7]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00


In [8]:
!pip install auto_gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
Discarding https://files.pythonhosted.org/packages/1b/79/5a3a7d877a9b0a72f528e9977ec65cdb9fad800fa4f5110f87f2acaaf6fe/auto_gptq-0.3.2.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto_gptq from https://files.pythonhosted.org/packages/1b/79/5a3a7d877a9b0a72f528e9977ec65cdb9fad800fa4f5110f87f2acaaf6fe/auto_gptq-0.3.2.tar.gz has inconsistent version: expected '0.3.2', but metadata has '0.3.2+cu118'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
Discarding https://files.pythonhosted.org/packages/3f/5c/28d57f833498e014e9d066ea0199a503b5e59d4c8e8e701a460223e143da/auto_gptq-0.3.1.tar.gz (from https://pypi.org/simple/auto-gptq/) (requires-python:>=3.8.0): Requested auto_gptq from https://files.pythonhosted.org/packages/3f/5c/28d57f833498e014e9d06

## Import the required packages

In [9]:
from transformers import AutoTokenizer,BitsAndBytesConfig, pipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Using small versions of bigger models
For this tutorial we will use the "starchat-beta" model. This model is based on another code generation model called “Starcoder”, but it has been improved for chat purposes.

Some other chat models, like chatgpt, have been trained with a special technique called RLHF, which makes them more robust and reliable. Starchat-beta has not been trained with this technique, so it might not work well in some situations. If you want to use this model for a critical project, you might want to look for other models that are ready for production.

Starchat-beta is a big model, which means it takes a long time to run. To make it faster, we will use a smaller version of it, called “starchat-beta-GPTQ”. This version has been compressed with a method called quantization, which reduces the size and the running time of the model. We will use a library called auto_gptq to load the compressed model and run it. You can find more information about the “starchat-beta-GPTQ” model [here](https://huggingface.co/TheBloke/starchat-beta-GPTQ).

The auto_gptq library is very useful for compressing big models and making them easier to use. It also works well with other libraries like transformers and accelerate, so you don’t have to change much of your code. If you want to learn more about the auto_gptq library, you can check out this [link](https://github.com/PanQiWei/AutoGPTQ).

In [10]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/starchat-beta-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        use_safetensors=True,
        device_map="auto",
        use_triton=False,
        quantize_config=None)

  0%|          | 0/965 [00:00<?, ?w/s]

The `AutoGPTQForCausalLM.from_quantized` function from the auto-gptq library allows you to load and use a quantized model you want from the huggingface hub if it's available or else you can load a local model as well. Since the auto-gptq library integrates seamlessly with the transformers library, it also has access to the different tricks provided by the accelerate library. The accelerate library distributes the model weights across your available devices (CPU memory, GPU memory and hard drive). This is done in order to make it possible for really big models to get loaded onto your small hardware.

To do this, you need to provide `device_map=“auto”`.
For more information on the "accelerate" library you can follow this [link](https://huggingface.co/docs/accelerate/index).

Sometimes we want our model to stop making texts when it has done its job. For example, if we ask it to find the answer to a question, we don’t want it to keep talking after it has given us the answer. To do this, we need to give our model a signal that tells it when to stop. This signal is called a stopping criteria.

A stopping criteria is a list of words that we choose. When our model sees one of these words, it knows it has finished and it stops. The words can be different depending on what we are asking the model to do. In this example, I have picked some words that I think are good signals for the model to stop.

Before we use the stopping criteria, we need to change the words into numbers that the model can understand. These numbers are called tokens.

In [11]:
stop_list = ['\n```\n', '\n¿']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[203, 914, 203], [203, 43647]]

We also need to convert the tensors into long-tensors.

In [12]:
import torch

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([203, 914, 203], device='cuda:0'),
 tensor([  203, 43647], device='cuda:0')]

Then, we will create a class by inheriting the `StoppingCriteria` class from `transformers`. This class would check when the tokens generated by the models matches our stop tokens and then will return true which can signal the transformers library to stop the generation.

In [14]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

## Using Huggingface models locally with Langchain
Langchain is a python library that enables you to create applications with large language models. It does all the heavy lifting while providing nice APIs to build powerful applications with LLMs.

The [official docs](https://docs.langchain.com/docs/) provide all the information you need about Langchain. So, do read them through to understand what else you can do with this awesome library.

We will now use the model object created in the previous section with Langchain. To do this, we will first create a “pipeline” using the pipeline function from the transformers library.

In [15]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer,
                stopping_criteria=stopping_criteria,
                max_new_tokens=60)

The model 'GPTBigCodeGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMH

We need to provide the type of action that we want to use the model for. Here it will be “text-generation”. Then we provide the model object and the tokenizer object which we created in the previous sections. Also, let’s not forget to provide the max_new_tokens parameter. This one controls the length of the texts that you want your model to generate. The bigger the number, the longer the generated texts and higher the inference time.

I have kept the number low to have some acceptable inference time but you can play around with the token length as much as your hardware allows.

Next, we need to import the langchain related libraries.

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import HuggingFacePipeline

We will use the HuggingFacePipeline class provided by Langchain. Here we ask Langchain to use the pipeline that we created in the previous section.

In [17]:
local_llm = HuggingFacePipeline(pipeline=pipe)

## Setting up the stage
We’re going to create a test script that will help us check if an ETL pipeline is working correctly. Here’s what’s happening: we have a pipeline that takes data from a source (a CSV file), transforms it, and then puts it in a source (a database). We want to carry on some tests for the data that is loaded onto the target database.

## The data
We’re going to use data from the [Game Recommendations on Steam](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam) dataset as our source data. Specifically, we’ll use the games.csv file. We’ll put this data into a sqlite database. Here’s the script I used to move the data from the source to the target. Don’t worry too much about the script itself - it’s not important for this tutorial.

In [18]:
import sqlite3
import pandas as pd

In [19]:
def df2sql(csv:str,table:str):
    df = pd.read_csv(csv)
    conn = sqlite3.connect("database.db")
    df.to_sql(table, con=conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()

In [20]:
df2sql('/kaggle/input/game-recommendations-on-steam/games.csv',
       'games')

Let’s take a look at our source and target data. We’ll use pandas to read data from the source csv file and to read data from the target sqlite table.

In [21]:
conn = sqlite3.connect("database.db")
select = """select * from games;"""

In [22]:
df1 = pd.read_sql_query(select, conn)

data from the target database converted into a dataframe.

In [23]:
df1.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,1,0,0,Very Positive,84,2199,9.99,9.99,0.0,1
1,22364,BRINK: Agents of Change,2011-08-03,1,0,0,Positive,85,21,2.99,2.99,0.0,1
2,113020,Monaco: What's Yours Is Mine,2013-04-24,1,1,1,Very Positive,92,3722,14.99,14.99,0.0,1
3,226560,Escape Dead Island,2014-11-18,1,0,0,Mixed,61,873,14.99,14.99,0.0,1
4,249050,Dungeon of the ENDLESS™,2014-10-27,1,1,0,Very Positive,88,8784,11.99,11.99,0.0,1


In [24]:
df1.shape

(50564, 13)

In [25]:
df2 =pd.read_csv('/kaggle/input/game-recommendations-on-steam/games.csv')

data from the source csv file.

In [26]:
df2.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


## Agent
Remember how we talked about using a language model to check some things on the data that we put into the new database? Let’s make a list of what we want to check.

1. Verify that the target data is the same as the data in the source.
2. Verify that the “win” column has numbers only.
3. Verify that the following games appear under the title column in the target data. games:"BRINK: Agents of Change", "Monaco: What's Yours Is Mine", "Escape Dead Island".

We have three options to do these tests.

1. Do them by hand. For example, use some tools like sql query to get the data from the old database, get the data from the new database and then look at them to see if your tests pass or fail.

2. Write some scripts to make the process easier. For example, a python script that would get the data from the old and new databases using sql queries and then check if a certain condition is true or false.

3. There is a new way that we can use because of large language models. Make an agent that would take an input in normal language that could be the test cases and then use a tool to see if the test condition passes or fails.

It is the third way that we are going to explore in upcoming sections and we will use something known as an agent to do this.

An agent is a model which has access to a set of tools. Depending on the user's input, this agent can decide which tools to use to accomplish the given task.

Since we are dealing with data here, we can use the pandas library as our tool of choice. Langchain has a pre-built agent which uses pandas as tool and takes action based on user input. 

You can build your own agent but that we will cover in some other video.

Let's begin by importing the required modules.

In [27]:
from langchain.agents import create_pandas_dataframe_agent

We will create a pandas agent provided by langchain. Invoking the pandas agent is easy as you just need to call the `create_pandas_dataframe_agent` api and pass your model, the dataframe alongwith other parameters. We set the `temperature` to 0 which prevents the randomness of the text generated by the model. We also provide the `early_stopping_method` and set it to "generate "which will instruct the model to do a final pass before stopping.

In [28]:
agent = create_pandas_dataframe_agent(local_llm, [df1, df2], verbose=True, 
                                      temperature=0, max_execution_time=1,early_stopping_method="generate")

Then we use the `agent.run` to feed our prompt to the modle and run inference.

In [29]:
agent.run("does the two dataframes have equal elements?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




> Entering new AgentExecutor chain...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Thought: I should compare the two dataframes to see if they have equal elements.
Action: python_repl_ast
Action Input:
```python
df1.equals(df2)
```

Observation: False
Thought:

Final Answer: No, the two dataframes do not have equal elements.



¿

> Finished chain.


'No, the two dataframes do not have equal elements.\n\n\n\n¿'

Everything works fine. So, let's scale it up to more number of tests.

In [30]:
tests = ["are the two dataframes equal?",
         "Verify that none of the rows in the first dataframe have blank values.",
         "Given the following column in the first dataframe. Verify that the column has integer datatype. Column: 'win'.",
        """Given the following string. Verify that the strings appear under the title column in the first dataframe. String:'BRINK: Agents of Change', 'Monaco: What's Yours Is Mine', 'Escape Dead Island'"""]

In [31]:
import asyncio
import time

In [32]:
agent = create_pandas_dataframe_agent(local_llm, [df1, df2], verbose=True,
                                      temperature=0, max_execution_time=1,early_stopping_method="generate")

We will run the `tests` list through a loop which will feed the tests to the agent and run inference sequentially.

In [33]:
s = time.perf_counter()
for test in tests:
    agent.run(test)
elapsed = time.perf_counter() - s
print(f"Serial executed in {elapsed:0.2f} seconds.")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




> Entering new AgentExecutor chain...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Thought: I should compare the two dataframes to see if they are equal.
Action: python_repl_ast
Action Input: df1 == df2
Observation:        app_id  title  date_release   win   mac  linux  rating  positive_ratio  \
0        True   True          True  True  True   True    True            True   
1        True   True          True  True  True   True    True            True   
2        True   True          True  True  True   True    True            True   
3        True   True          True  True  True   True    True            True   
4        True   True          True  True  True   True    True            True   
...       ...    ...           ...   ...   ...    ...     ...             ...   
50559    True   True          True  True  True   True    True            True   
50560    True   True          True  True  True   True    True            True   
50561    True   True          True  True  True   True    True            True   
50562    True   True          True  True  True   True    

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

Thought: I should check the data type of the column.
Action: python_repl_ast
Action Input:
```python
df1['win'].dtype
```

Observation: int64
Thought:

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



Final Answer: The 'win' column in the first dataframe has an integer data type.



¿

> Finished chain.


> Entering new AgentExecutor chain...


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Thought: I should use the index to select the rows
Action: python_repl_ast
Action Input: df1.loc[df1['title'].str.contains('BRINK: Agents of Change'), :]
Observation:    app_id                    title date_release  win  mac  linux    rating  \
1   22364  BRINK: Agents of Change   2011-08-03    1    0      0  Positive   

   positive_ratio  user_reviews  price_final  price_original  discount  \
1              85            21         2.99            2.99       0.0   

   steam_deck  
1           1  
Thought:
Final Answer: The strings appear under the title column in the first dataframe. String: 'BRINK: Agents of Change', 'Monaco: What's Yours Is Mine', 'Escape Dead Island'



¿

> Finished chain.
Serial executed in 72.49 seconds.


Look!we have actually created a very small test framework which runs through a given set of tests and let's an agent execute the tests for us with a given tool.

## Conclusion
Why does this matter? In the usual way of testing software, when we run a test script in a CI/CD system, it might happen that some tests fail by mistake because of some changes in the app we are testing. What if we could have a system that can run the tests for us with the tool we want and make decisions on its own with or without our help and then change its actions if the first action fails because of a change in the app. I like to call this self-healing tests.

There were some tries to make self-healing tests before, but it has never been this easy before big language models. This is the whole idea behind this part2 and the previous [part1](https://www.kaggle.com/code/sapal6/langchain-and-llms-for-automating-software-testing) of this series. My idea behind making these articles was to make a plan for making self-healing tests that can be used to make a framework to make “testing-agents” that can do software tests for us in a good way and fix themselves if needed. With this thought, I will end this article and this two part series, but I am planning more cool stuff, so keep an eye out for my content on my twitter handle @thecodingprojec and my youtube channel [The Datadrift](https://www.youtube.com/@thedatadrift).

Please upvote this notebook if you find this article helpful as this helps others to find this article.